In [1]:
import os
from stlf_torch_kit import  DataLoadeing
import torch
import pandas as pd
import numpy as np
import pickle, time
from stlf_torch_kit import univariate_multi_step
from stlf_torch_kit import SaveBestModel, PlotLossCurves, LoadModel, train, TestModel, BatchGenerator, results
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary


In [2]:
from platform import python_version
print(python_version())

3.7.0


# Data Loading

In [3]:
path_dataset =r'C:\Users\Personal\Desktop\students\AEP\AEP' #Edit
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.values
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [4]:
time_steps=24 #look back or sequence length, lag, window size #Edit
target_len = 1 #how much steps do you wana forecast #Edit
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=target_len)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=target_len)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=target_len)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.31334519386291504 sec


In [5]:
test_X.shape

(12105, 24, 21)

# Proposed Model

#### LSTM

In [5]:
# major edit
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.lstm = nn.LSTM(1, 20, 1, batch_first=True).to(self.device) # num-features, node/units, num-layers
        self.fc = nn.Linear(20, 1).to(self.device) #(in,out)

    def forward(self, x):
        out, _ = self.lstm(x) # _=(h,c)
        out = self.fc(out[:, -1, :]) 
        return out

In [6]:
model = LSTMModel()
print(summary(model, input_size=(64, 24, 1)))

Layer (type:depth-idx)                   Output Shape              Param #
LSTMModel                                [64, 1]                   --
├─LSTM: 1-1                              [64, 24, 20]              1,840
├─Linear: 1-2                            [64, 1]                   21
Total params: 1,861
Trainable params: 1,861
Non-trainable params: 0
Total mult-adds (M): 2.83
Input size (MB): 0.01
Forward/backward pass size (MB): 0.25
Params size (MB): 0.01
Estimated Total Size (MB): 0.26


#### CNN
<table style="border: 1px solid black; border-collapse: collapse; width: 100%;">
    <thead>
        <tr style="background-color: #f2f2f2;">
            <th style="border: 1px solid black; padding: 8px;">Layer</th>
            <th style="border: 1px solid black; padding: 8px;">Expected Input Shape</th>
            <th style="border: 1px solid black; padding: 8px;">Common Adjustments Needed</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="border: 1px solid black; padding: 8px;">Conv1d</td>
            <td style="border: 1px solid black; padding: 8px;">(batch_size, channels, sequence_length)</td>
            <td style="border: 1px solid black; padding: 8px;">Permute input from (batch, seq, feat) to (batch, feat, seq)</td>
        </tr>
        <tr>
            <td style="border: 1px solid black; padding: 8px;">LSTM</td>
            <td style="border: 1px solid black; padding: 8px;">(batch_size, sequence_length, features)</td>
            <td style="border: 1px solid black; padding: 8px;">No permutation if batch_first=True</td>
        </tr>
    </tbody>
</table>



In [7]:
class CNN1DModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3).to(self.device)
        self.fc = nn.Linear(32*22, 1).to(self.device) #32 channels ao 22 features dy chy kernel size 3 na bad kam shwal

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = torch.relu(self.conv1(x))  
        x = x.reshape(x.shape[0], -1)
        out = self.fc(x) 
        return out


In [8]:
model = CNN1DModel()
print(summary(model, input_size=(64, 24, 1)))
# summary(model, input_size=(64, 24, 1), col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Output Shape              Param #
CNN1DModel                               [64, 1]                   --
├─Conv1d: 1-1                            [64, 32, 22]              128
├─Linear: 1-2                            [64, 1]                   705
Total params: 833
Trainable params: 833
Non-trainable params: 0
Total mult-adds (M): 0.23
Input size (MB): 0.01
Forward/backward pass size (MB): 0.36
Params size (MB): 0.00
Estimated Total Size (MB): 0.37


In [6]:
class CNNLSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.conv1 = nn.Conv1d(in_channels=21, out_channels=64, kernel_size=3).to(self.device)
       
        self.lstm = nn.LSTM(64, 55, 3, batch_first=True).to(self.device)
        self.fc = nn.Linear(55, 25).to(self.device) 
        self.fc1 = nn.Linear(25, 1).to(self.device) 


    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = torch.relu(self.conv1(x))
        
        x = x.permute(0, 2, 1)
        out, _ = self.lstm(x) # _=(h,c)
        out = self.fc(out[:, -1, :])
        out = self.fc1(out)
        return out


In [7]:
model = CNNLSTMModel()
print(summary(model, input_size=(64, 24, 21)))

Layer (type:depth-idx)                   Output Shape              Param #
CNNLSTMModel                             [64, 1]                   --
├─Conv1d: 1-1                            [64, 64, 22]              4,096
├─LSTM: 1-2                              [64, 22, 55]              75,900
├─Linear: 1-3                            [64, 25]                  1,400
├─Linear: 1-4                            [64, 1]                   26
Total params: 81,422
Trainable params: 81,422
Non-trainable params: 0
Total mult-adds (M): 112.73
Input size (MB): 0.13
Forward/backward pass size (MB): 1.35
Params size (MB): 0.33
Estimated Total Size (MB): 1.81


#### Mostly Cited

In [60]:
class MOSTLY_CITED(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.lstm = nn.LSTM(1, 20, 2, batch_first=True).to(self.device)
        self.fc = nn.Linear(20, 1).to(self.device) 

    def forward(self, x):
        out, _ = self.lstm(x) # _=(h,c)
        out = torch.sigmoid(self.fc(out[:, -1, :]) )
        return out


In [8]:
'cuda' if torch.cuda.is_available() else 'cpu'

'cuda'

# instances

In [9]:
model

CNNLSTMModel(
  (conv1): Conv1d(21, 64, kernel_size=(3,), stride=(1,))
  (lstm): LSTM(64, 55, num_layers=3, batch_first=True)
  (fc): Linear(in_features=55, out_features=25, bias=True)
  (fc1): Linear(in_features=25, out_features=1, bias=True)
)

In [10]:
model = CNNLSTMModel()#Edit
criterion = nn.MSELoss() #Edit, don't change

save_best_model = SaveBestModel()
Plot_Loss=PlotLossCurves()
load_model=LoadModel()

# Learning Rate & Optimizer

In [11]:
lr=0.001 # Edit
optimizer = torch.optim.Adam(model.parameters(), lr=lr) #Edit

# Check Device

In [12]:
def get_model_device(model):
    return next(model.parameters()).device
device = get_model_device(model)
print("Model is on device:", device)

Model is on device: cuda:0


# Training

#### Path & other Stuffs

In [13]:
start_epoch = 1
num_epochs = 20 #Edit
best_model_path=r'D:\chk6'+str('\\') #Edit
fig_path=r'D:\chk6' #Edit
train_data_loader, validation_data_loader, test_data_loader = DataLoadeing(train_X ,
                                                                           train_y, 
                                                                           validation_X, 
                                                                           validation_y, 
                                                                           test_X, 
                                                                           test_y, 
                                                                           batch_size=32) #Batch_Size Edit

#### Instances

In [14]:
criterion = nn.MSELoss() # Edit, for Now Don't  Change
save_best_model = SaveBestModel()
Plot_Loss=PlotLossCurves()
load_model=LoadModel()

#### Training Loop

In [15]:
start = time.time()
train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader)
print('Time Consumed', time.time()-start, "sec")

Epoch [1/20], Step [2653/2653], Training Loss: 0.0092
Epoch [1/20], Step [758/758], Val Loss: 0.0109

Saving best model for epoch: 1
 at D:\chk6\1best_model.pth
Epoch [2/20], Step [2653/2653], Training Loss: 0.0043
Epoch [2/20], Step [758/758], Val Loss: 0.0052

Saving best model for epoch: 2
 at D:\chk6\2best_model.pth
Epoch [3/20], Step [2653/2653], Training Loss: 0.0010
Epoch [3/20], Step [758/758], Val Loss: 0.0032

Saving best model for epoch: 3
 at D:\chk6\3best_model.pth
Epoch [4/20], Step [2653/2653], Training Loss: 0.0004
Epoch [4/20], Step [758/758], Val Loss: 0.0005

Saving best model for epoch: 4
 at D:\chk6\4best_model.pth
Epoch [5/20], Step [2653/2653], Training Loss: 0.0003
Epoch [5/20], Step [758/758], Val Loss: 0.0003

Saving best model for epoch: 5
 at D:\chk6\5best_model.pth
Epoch [6/20], Step [2653/2653], Training Loss: 0.0002
Epoch [6/20], Step [758/758], Val Loss: 0.0003
Epoch [7/20], Step [2653/2653], Training Loss: 0.0002
Epoch [7/20], Step [758/758], Val Loss: 

#### Results

In [19]:
import torch
torch.cuda.empty_cache()

In [16]:
load_model_path=r'D:\chk6\10best_model.pth' # Edit
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

# MAPE, MAE, RMSE

New lr = 0.001
Time Consumed 0.8097715377807617 sec
Mean Absolute Error (MAE): 152.4
Median Absolute Error (MedAE): 120.78
Mean Squared Error (MSE): 40403.06
Root Mean Squared Error (RMSE): 201.01
Mean Absolute Percentage Error (MAPE): 1.04 %
Median Absolute Percentage Error (MDAPE): 0.83 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


# Fine Tunning 1

In [17]:
num_epochs = 100
lr=0.0001
Batch_size = True #batch size maintained

#### Load Model for tuning

In [18]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=32,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 0.0001


In [19]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [11/110], Step [2653/2653], Training Loss: 0.0001
Epoch [11/110], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 11
 at D:\chk6\11best_model.pth
Epoch [12/110], Step [2653/2653], Training Loss: 0.0001
Epoch [12/110], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 12
 at D:\chk6\12best_model.pth
Epoch [13/110], Step [2653/2653], Training Loss: 0.0001
Epoch [13/110], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 13
 at D:\chk6\13best_model.pth
Epoch [14/110], Step [2653/2653], Training Loss: 0.0001
Epoch [14/110], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 14
 at D:\chk6\14best_model.pth
Epoch [15/110], Step [2653/2653], Training Loss: 0.0001
Epoch [15/110], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 15
 at D:\chk6\15best_model.pth
Epoch [16/110], Step [2653/2653], Training Loss: 0.0001
Epoch [16/110], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 16
 at D:\chk6\16best_model.pt

Epoch [63/110], Step [2653/2653], Training Loss: 0.0001
Epoch [63/110], Step [758/758], Val Loss: 0.0001
Epoch [64/110], Step [2653/2653], Training Loss: 0.0001
Epoch [64/110], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 64
 at D:\chk6\64best_model.pth
Epoch [65/110], Step [2653/2653], Training Loss: 0.0001
Epoch [65/110], Step [758/758], Val Loss: 0.0001
Epoch [66/110], Step [2653/2653], Training Loss: 0.0001
Epoch [66/110], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 66
 at D:\chk6\66best_model.pth
Epoch [67/110], Step [2653/2653], Training Loss: 0.0001
Epoch [67/110], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 67
 at D:\chk6\67best_model.pth
Epoch [68/110], Step [2653/2653], Training Loss: 0.0001
Epoch [68/110], Step [758/758], Val Loss: 0.0001
Epoch [69/110], Step [2653/2653], Training Loss: 0.0001
Epoch [69/110], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 69
 at D:\chk6\69best_model.pth
Epoch [70/110], S

In [20]:
load_model_path=r'D:\chk6\88best_model.pth'
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 0.0001
Time Consumed 0.33310914039611816 sec
Mean Absolute Error (MAE): 91.52
Median Absolute Error (MedAE): 73.46
Mean Squared Error (MSE): 14804.34
Root Mean Squared Error (RMSE): 121.67
Mean Absolute Percentage Error (MAPE): 0.62 %
Median Absolute Percentage Error (MDAPE): 0.51 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


# Fine Tunning 2

In [24]:
num_epochs = 20
lr = 0.00001
Batch_size=32
load_model_path=r'D:\chk6\88best_model.pth'

In [25]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [26]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [89/108], Step [2653/2653], Training Loss: 0.0001
Epoch [89/108], Step [758/758], Val Loss: 0.0001
Epoch [90/108], Step [2653/2653], Training Loss: 0.0001
Epoch [90/108], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 90
 at D:\chk6\90best_model.pth
Epoch [91/108], Step [2653/2653], Training Loss: 0.0001
Epoch [91/108], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 91
 at D:\chk6\91best_model.pth
Epoch [92/108], Step [2653/2653], Training Loss: 0.0001
Epoch [92/108], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 92
 at D:\chk6\92best_model.pth
Epoch [93/108], Step [2653/2653], Training Loss: 0.0000
Epoch [93/108], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 93
 at D:\chk6\93best_model.pth
Epoch [94/108], Step [2653/2653], Training Loss: 0.0000
Epoch [94/108], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 94
 at D:\chk6\94best_model.pth
Epoch [95/108], Step [2653/2653], Training Loss: 0.0000
Epoc

In [27]:
load_model_path=r'D:\chk6\108best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.29328155517578125 sec
Mean Absolute Error (MAE): 91.01
Median Absolute Error (MedAE): 71.54
Mean Squared Error (MSE): 14858.5
Root Mean Squared Error (RMSE): 121.9
Mean Absolute Percentage Error (MAPE): 0.62 %
Median Absolute Percentage Error (MDAPE): 0.5 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [28]:
num_epochs = 40
#lr = 0.0001
Batch_size=32
load_model_path=r'D:\chk6\108best_model.pth'

In [29]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [30]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [109/148], Step [2653/2653], Training Loss: 0.0000
Epoch [109/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 109
 at D:\chk6\109best_model.pth
Epoch [110/148], Step [2653/2653], Training Loss: 0.0000
Epoch [110/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 110
 at D:\chk6\110best_model.pth
Epoch [111/148], Step [2653/2653], Training Loss: 0.0000
Epoch [111/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 111
 at D:\chk6\111best_model.pth
Epoch [112/148], Step [2653/2653], Training Loss: 0.0000
Epoch [112/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 112
 at D:\chk6\112best_model.pth
Epoch [113/148], Step [2653/2653], Training Loss: 0.0000
Epoch [113/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 113
 at D:\chk6\113best_model.pth
Epoch [114/148], Step [2653/2653], Training Loss: 0.0000
Epoch [114/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 114
 at 

In [31]:
load_model_path=r'D:\chk6\148best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.7108466625213623 sec
Mean Absolute Error (MAE): 90.44
Median Absolute Error (MedAE): 70.79
Mean Squared Error (MSE): 14699.12
Root Mean Squared Error (RMSE): 121.24
Mean Absolute Percentage Error (MAPE): 0.62 %
Median Absolute Percentage Error (MDAPE): 0.49 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [32]:
num_epochs = 40
#lr = 0.00001
Batch_size=32
load_model_path=r'D:\chk6\148best_model.pth'

In [33]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [34]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [149/188], Step [2653/2653], Training Loss: 0.0000
Epoch [149/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 149
 at D:\chk6\149best_model.pth
Epoch [150/188], Step [2653/2653], Training Loss: 0.0000
Epoch [150/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 150
 at D:\chk6\150best_model.pth
Epoch [151/188], Step [2653/2653], Training Loss: 0.0000
Epoch [151/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 151
 at D:\chk6\151best_model.pth
Epoch [152/188], Step [2653/2653], Training Loss: 0.0000
Epoch [152/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 152
 at D:\chk6\152best_model.pth
Epoch [153/188], Step [2653/2653], Training Loss: 0.0000
Epoch [153/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 153
 at D:\chk6\153best_model.pth
Epoch [154/188], Step [2653/2653], Training Loss: 0.0000
Epoch [154/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 154
 at 

In [35]:
load_model_path=r'D:\chk6\188best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.32709288597106934 sec
Mean Absolute Error (MAE): 90.16
Median Absolute Error (MedAE): 70.6
Mean Squared Error (MSE): 14619.34
Root Mean Squared Error (RMSE): 120.91
Mean Absolute Percentage Error (MAPE): 0.61 %
Median Absolute Percentage Error (MDAPE): 0.49 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [36]:
num_epochs = 60
#lr = 0.0001
Batch_size=32
load_model_path=r'D:\chk6\188best_model.pth'

In [37]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [38]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [189/248], Step [2653/2653], Training Loss: 0.0000
Epoch [189/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 189
 at D:\chk6\189best_model.pth
Epoch [190/248], Step [2653/2653], Training Loss: 0.0000
Epoch [190/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 190
 at D:\chk6\190best_model.pth
Epoch [191/248], Step [2653/2653], Training Loss: 0.0000
Epoch [191/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 191
 at D:\chk6\191best_model.pth
Epoch [192/248], Step [2653/2653], Training Loss: 0.0000
Epoch [192/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 192
 at D:\chk6\192best_model.pth
Epoch [193/248], Step [2653/2653], Training Loss: 0.0000
Epoch [193/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 193
 at D:\chk6\193best_model.pth
Epoch [194/248], Step [2653/2653], Training Loss: 0.0000
Epoch [194/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 194
 at 

Epoch [238/248], Step [758/758], Val Loss: 0.0001
Epoch [239/248], Step [2653/2653], Training Loss: 0.0000
Epoch [239/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 239
 at D:\chk6\239best_model.pth
Epoch [240/248], Step [2653/2653], Training Loss: 0.0000
Epoch [240/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 240
 at D:\chk6\240best_model.pth
Epoch [241/248], Step [2653/2653], Training Loss: 0.0000
Epoch [241/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 241
 at D:\chk6\241best_model.pth
Epoch [242/248], Step [2653/2653], Training Loss: 0.0000
Epoch [242/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 242
 at D:\chk6\242best_model.pth
Epoch [243/248], Step [2653/2653], Training Loss: 0.0000
Epoch [243/248], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 243
 at D:\chk6\243best_model.pth
Epoch [244/248], Step [2653/2653], Training Loss: 0.0000
Epoch [244/248], Step [758/758], Val L

In [39]:
load_model_path=r'D:\chk6\248best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.3287696838378906 sec
Mean Absolute Error (MAE): 89.88
Median Absolute Error (MedAE): 70.36
Mean Squared Error (MSE): 14538.16
Root Mean Squared Error (RMSE): 120.57
Mean Absolute Percentage Error (MAPE): 0.61 %
Median Absolute Percentage Error (MDAPE): 0.49 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [147]:
num_epochs = 60
#lr = 0.0001
Batch_size=32
load_model_path=r'D:\chk5\260best_model.pth'

In [148]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [ ]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [261/320], Step [2653/2653], Training Loss: 0.0000
Epoch [261/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 261
 at D:\chk5\261best_model.pth
Epoch [262/320], Step [2653/2653], Training Loss: 0.0000
Epoch [262/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 262
 at D:\chk5\262best_model.pth
Epoch [263/320], Step [2653/2653], Training Loss: 0.0000
Epoch [263/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 263
 at D:\chk5\263best_model.pth
Epoch [264/320], Step [2653/2653], Training Loss: 0.0000
Epoch [264/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 264
 at D:\chk5\264best_model.pth
Epoch [265/320], Step [2653/2653], Training Loss: 0.0000
Epoch [265/320], Step [758/758], Val Loss: 0.0001
Epoch [266/320], Step [2653/2653], Training Loss: 0.0000
Epoch [266/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 266
 at D:\chk5\266best_model.pth
Epoch [267/320], Step [2653/2653], Tra

Epoch [310/320], Step [2653/2653], Training Loss: 0.0000
Epoch [310/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 310
 at D:\chk5\310best_model.pth
Epoch [311/320], Step [2653/2653], Training Loss: 0.0000
Epoch [311/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 311
 at D:\chk5\311best_model.pth
Epoch [312/320], Step [2653/2653], Training Loss: 0.0000
Epoch [312/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 312
 at D:\chk5\312best_model.pth
Epoch [313/320], Step [2653/2653], Training Loss: 0.0000
Epoch [313/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 313
 at D:\chk5\313best_model.pth
Epoch [314/320], Step [2653/2653], Training Loss: 0.0000
Epoch [314/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 314
 at D:\chk5\314best_model.pth
Epoch [315/320], Step [2653/2653], Training Loss: 0.0000
Epoch [315/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 315
 at 

In [12]:
lr = 0.00001

In [13]:
load_model_path=r'D:\chk5\318best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.39956188201904297 sec
Mean Absolute Error (MAE): 89.36
Median Absolute Error (MedAE): 69.47
Mean Squared Error (MSE): 14358.5
Root Mean Squared Error (RMSE): 119.83
Mean Absolute Percentage Error (MAPE): 0.61 %
Median Absolute Percentage Error (MDAPE): 0.48 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [14]:
best_model_path=r'D:\chk5'+str('\\') #Edit
fig_path=r'D:\chk5'

In [15]:
num_epochs = 60
#lr = 0.0001
Batch_size=32
load_model_path=r'D:\chk5\260best_model.pth'

In [16]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [17]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [261/320], Step [2653/2653], Training Loss: 0.0000
Epoch [261/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 261
 at D:\chk5\261best_model.pth
Epoch [262/320], Step [2653/2653], Training Loss: 0.0000
Epoch [262/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 262
 at D:\chk5\262best_model.pth
Epoch [263/320], Step [2653/2653], Training Loss: 0.0000
Epoch [263/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 263
 at D:\chk5\263best_model.pth
Epoch [264/320], Step [2653/2653], Training Loss: 0.0000
Epoch [264/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 264
 at D:\chk5\264best_model.pth
Epoch [265/320], Step [2653/2653], Training Loss: 0.0000
Epoch [265/320], Step [758/758], Val Loss: 0.0001
Epoch [266/320], Step [2653/2653], Training Loss: 0.0000
Epoch [266/320], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 266
 at D:\chk5\266best_model.pth
Epoch [267/320], Step [2653/2653], Tra

In [18]:
load_model_path=r'D:\chk5\320best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.31932806968688965 sec
Mean Absolute Error (MAE): 89.35
Median Absolute Error (MedAE): 69.52
Mean Squared Error (MSE): 14356.16
Root Mean Squared Error (RMSE): 119.82
Mean Absolute Percentage Error (MAPE): 0.61 %
Median Absolute Percentage Error (MDAPE): 0.49 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [19]:
num_epochs = 60
#lr = 0.0001
Batch_size=32
load_model_path=r'D:\chk5\320best_model.pth'

In [20]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [ ]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [321/380], Step [2653/2653], Training Loss: 0.0000
Epoch [321/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 321
 at D:\chk5\321best_model.pth
Epoch [322/380], Step [2653/2653], Training Loss: 0.0000
Epoch [322/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 322
 at D:\chk5\322best_model.pth
Epoch [323/380], Step [2653/2653], Training Loss: 0.0000
Epoch [323/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 323
 at D:\chk5\323best_model.pth
Epoch [324/380], Step [2653/2653], Training Loss: 0.0000
Epoch [324/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 324
 at D:\chk5\324best_model.pth
Epoch [325/380], Step [2653/2653], Training Loss: 0.0000
Epoch [325/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 325
 at D:\chk5\325best_model.pth
Epoch [326/380], Step [2653/2653], Training Loss: 0.0000
Epoch [326/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 326
 at 

Epoch [369/380], Step [2653/2653], Training Loss: 0.0000
Epoch [369/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 369
 at D:\chk5\369best_model.pth
Epoch [370/380], Step [2653/2653], Training Loss: 0.0000
Epoch [370/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 370
 at D:\chk5\370best_model.pth
Epoch [371/380], Step [2653/2653], Training Loss: 0.0000
Epoch [371/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 371
 at D:\chk5\371best_model.pth
Epoch [372/380], Step [2653/2653], Training Loss: 0.0000
Epoch [372/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 372
 at D:\chk5\372best_model.pth
Epoch [373/380], Step [2653/2653], Training Loss: 0.0000
Epoch [373/380], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 373
 at D:\chk5\373best_model.pth
Epoch [374/380], Step [2653/2653], Training Loss: 0.0000
Epoch [374/380], Step [758/758], Val Loss: 0.0001
Epoch [375/380], Step [2653/2653], Tra

In [10]:
lr = 0.00001

In [11]:
load_model_path=r'D:\chk5\376best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.43485236167907715 sec
Mean Absolute Error (MAE): 89.15
Median Absolute Error (MedAE): 69.46
Mean Squared Error (MSE): 14294.5
Root Mean Squared Error (RMSE): 119.56
Mean Absolute Percentage Error (MAPE): 0.61 %
Median Absolute Percentage Error (MDAPE): 0.48 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [12]:
best_model_path=r'D:\chk5'+str('\\') #Edit
fig_path=r'D:\chk5'

In [17]:
num_epochs = 60
#lr = 0.0001
Batch_size=32
load_model_path=r'D:\chk5\436best_model.pth'

In [18]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [19]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [437/496], Step [2653/2653], Training Loss: 0.0000
Epoch [437/496], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 437
 at D:\chk5\437best_model.pth
Epoch [438/496], Step [2653/2653], Training Loss: 0.0000
Epoch [438/496], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 438
 at D:\chk5\438best_model.pth
Epoch [439/496], Step [2653/2653], Training Loss: 0.0000
Epoch [439/496], Step [758/758], Val Loss: 0.0001
Epoch [440/496], Step [2653/2653], Training Loss: 0.0000
Epoch [440/496], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 440
 at D:\chk5\440best_model.pth
Epoch [441/496], Step [2653/2653], Training Loss: 0.0000
Epoch [441/496], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 441
 at D:\chk5\441best_model.pth
Epoch [442/496], Step [2653/2653], Training Loss: 0.0000
Epoch [442/496], Step [758/758], Val Loss: 0.0001
Epoch [443/496], Step [2653/2653], Training Loss: 0.0000
Epoch [443/496], Step [758/758], Val Loss: 0.

In [20]:
load_model_path=r'D:\chk5\496best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.2789039611816406 sec
Mean Absolute Error (MAE): 88.81
Median Absolute Error (MedAE): 69.05
Mean Squared Error (MSE): 14186.4
Root Mean Squared Error (RMSE): 119.11
Mean Absolute Percentage Error (MAPE): 0.61 %
Median Absolute Percentage Error (MDAPE): 0.48 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [27]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [2]:
pip install torchinfo

  Using cached torchinfo-1.8.0-py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.
